In [1]:
import os

os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from dask.diagnostics import ProgressBar
from pathlib import Path
import libs.utils
import libs.vars
import numpy as np
import xarray

xarray.set_options(keep_attrs=True)

def create_siconc_weighted_var(ensemble, files, component, variable_id):
    variable_id_new = f'{variable_id}_area'

    for i, item in enumerate(ensemble):
        source_id = item['source_id']
        variant_label = item['variant_label']
        
        grid_label = 'gn'
        #if source_id == 'EC-Earth3':
        #    grid_label = 'gr'

        format_vars = {
            'component': component,
            'grid_label': grid_label,
            'source_id': source_id,
            'variant_label': variant_label
        }
       
        format_siconc_vars = {
            'component': 'SImon',
            'grid_label': 'gn',
            'source_id': source_id,
            'variant_label': variant_label
        }
        
        for filename in files:            
            base_path = f'_data/cmip6/{source_id}/{{variable_id}}/'
            filepath_var = (base_path + filename).format(variable_id=variable_id, **format_vars)
            if not Path(filepath_var).exists():
                continue

            filepath_siconc = (base_path + filename).format(variable_id='siconc', **format_siconc_vars)
            filepath_var_new = (base_path + filename).format(variable_id=variable_id_new, **format_vars)
            
            print('Input:')
            print('->', filepath_var)
            
            print('Weight:')
            print('->', filepath_siconc)

            data_var = xarray.open_mfdataset(paths=filepath_var, combine='by_coords', use_cftime=True)
                
            data_siconc = xarray.open_mfdataset(paths=filepath_siconc, combine='by_coords', use_cftime=True)

            data_var[variable_id] = data_var[variable_id].where(data_var[variable_id].latitude > 60)
            data_var[variable_id] *= data_siconc.siconc
            data_var[variable_id] /= 100 # Convert from % to frac for siconc, nb simpconc handled later

            data_var = data_var.rename({ (variable_id): variable_id_new })

            Path(filepath_var_new).parent.mkdir(parents=True, exist_ok=True)
            print('Output:')
            print('->', filepath_var_new)

            write = data_var.to_netcdf(
                filepath_var_new,
                compute=False,
                engine='netcdf4',
                unlimited_dims=['time']
            )
            with ProgressBar():
                write.compute()

            data_var.close()
            data_siconc.close()
            print('-> Saved to disk')

            # Finally, compress as to_netcdf() seems to produce large file sizes
            libs.utils.compress_nc_file(filepath_var_new, filepath_var_new)
            print('-> Compressed')
            print('-' * 20)


ensemble = libs.vars.ensemble()
files = [
    f'{{variable_id}}_{{component}}_{{source_id}}_historical_{{variant_label}}_{{grid_label}}_198001-201412_processed.nc',
    f'{{variable_id}}_{{component}}_{{source_id}}_ssp585_{{variant_label}}_{{grid_label}}_201501-210012_processed.nc'
]
variables = [
    { 'variable_id': 'simpconc', 'component': 'SImon' }
]

for item in variables:
    create_siconc_weighted_var(ensemble, files, **item)

Input:
-> _data/cmip6/UKESM1-0-LL/simpconc/simpconc_SImon_UKESM1-0-LL_historical_r2i1p1f2_gn_198001-201412_processed.nc
Weight:
-> _data/cmip6/UKESM1-0-LL/siconc/siconc_SImon_UKESM1-0-LL_historical_r2i1p1f2_gn_198001-201412_processed.nc
Output:
-> _data/cmip6/UKESM1-0-LL/simpconc_area/simpconc_area_SImon_UKESM1-0-LL_historical_r2i1p1f2_gn_198001-201412_processed.nc
[########################################] | 100% Completed |  3.7s
-> Saved to disk
-> Compressed
--------------------
Input:
-> _data/cmip6/UKESM1-0-LL/simpconc/simpconc_SImon_UKESM1-0-LL_ssp585_r2i1p1f2_gn_201501-210012_processed.nc
Weight:
-> _data/cmip6/UKESM1-0-LL/siconc/siconc_SImon_UKESM1-0-LL_ssp585_r2i1p1f2_gn_201501-210012_processed.nc
Output:
-> _data/cmip6/UKESM1-0-LL/simpconc_area/simpconc_area_SImon_UKESM1-0-LL_ssp585_r2i1p1f2_gn_201501-210012_processed.nc
[########################################] | 100% Completed |  9.0s
-> Saved to disk
-> Compressed
--------------------
Input:
-> _data/cmip6/NorESM2-LM/sim